<a href="https://colab.research.google.com/github/modichirag/flowpm/blob/tfcosmo/notebooks/my_flowpm_blog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet  git+https://github.com/modichirag/flowpm.git@blogpost;
!git clone --quiet -b blogpost https://github.com/modichirag/flowpm.git;

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import InterpolatedUnivariateSpline as iuspline
%matplotlib inline
from flowpm import linear_field, lpt_init, nbody, cic_paint
import flowpm

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
## Define key parameters of our Universe
a0      = 0.1       #the initial epoch from where we begin the evolution, a0=0 corresponds to birth but is numerically unstable
af      = 1.0       #final epoch where we visualize structures, af=1 corresponds to today
n_steps = 30        #number of time-steps to split the total evolution into
L       = 100       #Physical size of the Universe in Mpc/h
N       = 64        #Number of mesh-points along one axis, size of the cube. Then the number of particles will be N^3
batch   = 2         #Batch size, how many independent Universes to simulate

stages = np.linspace(a0, af, n_steps, endpoint=True) #time-steps for the integration

#Read-in the linear power-spectrum from a file. 
#This is determined by some fundamental parameters of our Universe that cosmologists are interested in
#In N-body simulations, this determines the clustering of the Universe at the initial epoch a0
klin = np.loadtxt('flowpm/flowpm/data/Planck15_a1p00.txt').T[0]
plin = np.loadtxt('flowpm/flowpm/data/Planck15_a1p00.txt').T[1]
ipklin = iuspline(klin, plin)

In [4]:
tf.reset_default_graph()
# Generate Gaussian initial conditions for matter density 
initial_conditions = flowpm.linear_field(N,  L, ipklin, batch_size=batch)   

# Sample particles, i.e. generate the initial displacement and velocity
# State is a tensor of shape (3, batch, N^3, 3)
# where the last axiss corresponds to the 3: x, y, z directions 
# and the first axis corresponds to position, velocity, acceleration of particle
state = flowpm.lpt_init(initial_conditions, a0=a0)   

# Evolve particles from initial state down to a=af
final_state = flowpm.nbody(state, stages, N)         

# Retrieve final density field i.e interpolate the particles to the mesh
final_field = flowpm.cic_paint(tf.zeros_like(initial_conditions), final_state[0])

TypeError: unsupported operand type(s) for /: 'Dimension' and 'int', please use // instead